In [ ]:
import pandas as pd
import pyarrow
from sqlalchemy import create_engine
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()

In [ ]:
try:
    # Conexión a SalesSystem (Facturación)
    salessystem_url = f"{os.getenv('DB_SALESSYSTEM_DIALECT')}://{os.getenv('DB_SALESSYSTEM_USER')}:{os.getenv('DB_SALESSYSTEM_PASSWORD')}@{os.getenv('DB_SALESSYSTEM_HOST')}:{os.getenv('DB_SALESSYSTEM_PORT')}/{os.getenv('DB_SALESSYSTEM_NAME')}"
    salessystem = create_engine(salessystem_url)
    
    # Conexión a Warehouse (Contabilidad)
    warehouse_url = f"{os.getenv('DB_WAREHOUSE_DIALECT')}://{os.getenv('DB_WAREHOUSE_USER')}:{os.getenv('DB_WAREHOUSE_PASSWORD')}@{os.getenv('DB_WAREHOUSE_HOST')}:{os.getenv('DB_WAREHOUSE_PORT')}/{os.getenv('DB_WAREHOUSEE_NAME')}"
    warehouse = create_engine(warehouse_url)
    
    # Validar las conexiones
    with salessystem.connect() as conn_sales:
        pass
    with warehouse.connect() as conn_warehouse:
        pass
        
except Exception as e:
    print(f"Error al conectar a las bases de datos: {str(e)}")
    raise

In [3]:
pedidos=pd.read_sql('SELECT * FROM pedidos', salessystem)
cotizaciones=pd.read_sql('SELECT * FROM facturas', salessystem)
proveedores=pd.read_sql('SELECT * FROM proveedores', salessystem)
adquirientes=pd.read_sql('SELECT * FROM customers', salessystem)
guias=pd.read_sql('SELECT * FROM remision_remitente', salessystem)
vehiculos=pd.read_sql('SELECT * FROM vehiculos', salessystem)
bcp=pd.read_sql('SELECT * FROM v_bcp', salessystem)
info=pd.read_sql('SELECT * FROM info', salessystem)
catalogo=pd.read_sql('SELECT * FROM catalogo', salessystem)
ibk=pd.read_sql('SELECT * FROM v_ibk', salessystem)
acc_7=pd.read_sql('SELECT * FROM acc._7', warehouse)
acc_5=pd.read_sql('SELECT * FROM acc._5', warehouse)
priv_entities=pd.read_sql('SELECT * FROM priv.entities', warehouse)

**PEDIDOS EN CURSO**

In [4]:
df1_filtrado=pedidos.loc[pedidos['estado'].isin(['PENDIENTE', 'EN PROCESO'])]
df1=pd.merge(df1_filtrado,adquirientes,left_on='adquiriente', right_on='ruc', how='left')
df1['alias'] = df1['alias'].fillna(df1['adquiriente'].astype(str))
df1_ordenado = df1.sort_values(by=['periodo', 'related_user', 'alias'])
df1=df1_ordenado[['periodo','related_user','alias', 'importe_total', 'cod_pedido', 'estado']]
df1

,periodo,related_user,alias,importe_total,cod_pedido,estado
17,202505,AARON,FENIX,20000.00,R134FF9D1,EN PROCESO
30,202505,AARON,NEYRA C,6500.00,X134FFA27,EN PROCESO
34,202505,AARON,TRANSTIK,60000.00,X134FFA32,EN PROCESO
1,202505,ARELLANO,ECONO GROUP,10000.00,R134FF922,EN PROCESO
18,202505,ARELLANO,INNOVA,8000.00,X134FFA01,EN PROCESO
0,202505,ARELLANO,JUANDY,30000.00,R134FF921,EN PROCESO
16,202505,ARELLANO,PERUGRAF,12000.00,R134FF9C4,EN PROCESO
24,202505,ARELLANO,PETITO,3000.00,X134FFA21,EN PROCESO
15,202505,EDGAR,BAUTISTA,10000.00,R134FF9C3,EN PROCESO
11,202505,EDGAR,NIMA J,46000.00,R134FF9B1,EN PROCESO


**COMPARACION PEDIDOS VS TOTAL DE COTIZACIONES INGRESADAS**

In [5]:
df2 = cotizaciones.loc[cotizaciones['cod_pedido'].isin(df1['cod_pedido'].tolist())].copy()  # Crea una copia explícita
df2.loc[:, 'total'] = round((df2['cantidad'].astype(float) * df2['precio_unit'].astype(float)) * 1.18, 0).astype(int)
df2_totales = df2.groupby('cod_pedido')['total'].sum().reset_index()
df2_comparado=pd.merge(df2_totales,df1,on='cod_pedido', how='left')
df2_comparado = df2_comparado.rename(columns={'importe_total': 'pedido', 'total': 'cotizaciones'})
df2_comparado.loc[:,'diferencia']=df2_comparado['pedido']-df2_comparado['cotizaciones']
df1_ordenado = df2_comparado.sort_values(by=['periodo', 'related_user', 'alias'])
df2_ordenado=df1_ordenado[['periodo', 'related_user', 'alias', 'pedido', 'cotizaciones', 'diferencia', 'cod_pedido', 'estado']]
df2_ordenado

,periodo,related_user,alias,pedido,cotizaciones,diferencia,cod_pedido,estado
21,202505,AARON,FENIX,20000.00,20000,0.00,R134FF9D1,EN PROCESO
34,202505,AARON,NEYRA C,6500.00,6501,-1.00,X134FFA27,EN PROCESO
37,202505,AARON,TRANSTIK,60000.00,59999,1.00,X134FFA32,EN PROCESO
5,202505,ARELLANO,ECONO GROUP,10000.00,10000,0.00,R134FF922,EN PROCESO
22,202505,ARELLANO,INNOVA,8000.00,8000,0.00,X134FFA01,EN PROCESO
4,202505,ARELLANO,JUANDY,30000.00,41502,-11502.00,R134FF921,EN PROCESO
20,202505,ARELLANO,PERUGRAF,12000.00,12000,0.00,R134FF9C4,EN PROCESO
28,202505,ARELLANO,PETITO,3000.00,3001,-1.00,X134FFA21,EN PROCESO
19,202505,EDGAR,BAUTISTA,10000.00,10000,0.00,R134FF9C3,EN PROCESO
15,202505,EDGAR,NIMA J,46000.00,46002,-2.00,R134FF9B1,EN PROCESO


**DESCRIPCION POR PEDIDO**

In [6]:
cod_pedido='X134FED84'
df3_filtrado=cotizaciones.loc[cotizaciones['cod_pedido'] == 'X134FED84']
cant_facturas = len(df3_filtrado['cuo'].drop_duplicates())
"""
INPUT CODIGO DE PEDIDO
FILTRAR COTIZACIONES DE ESE CODIGO DE PEDIDO
CALCULAR CANTIDAD DE COTIZACIONES DEL PEDIDO

"""


'\nINPUT CODIGO DE PEDIDO\nFILTRAR COTIZACIONES DE ESE CODIGO DE PEDIDO\nCALCULAR CANTIDAD DE COTIZACIONES DEL PEDIDO\n\n'

#SELECT
i.transaccion as transaccion,
DATE_FORMAT(i.fecha,'%d/%m/%Y') as fecha,
i.hora as hora,
LPAD(operacion, 5, '0') as operacion,
SUBSTRING(proveedores.nombre_razon, 1, 8) as nombre,
'CUENTA  CORRIENTE  MN' as tipo_cuenta, #doble espacio
info.ibk as nro_cuenta,
info.cci_ibk as cci,
FORMAT(importe, 2) as importe,
FORMAT(importe, 2) as efectivo,
FORMAT(importe, 2) as recibido_dm,
FORMAT(importe, 2) as recibido_um,
IF(importe > 9999.99, 'False', 'True') as recibido_um_v,
IF(importe < 10000, 'False', 'True') as recibido_dm_v,
FORMAT(importe, 2) as redondeado,
IF(importe > 9999.99, 'False', 'True') as pre_recibido_um,
IF(importe < 10000, 'False', 'True') as pre_recibido_dm,
REPLACE(i.ejecutante, ' ', '  ') as ejecutante #doble espacio por cada espacio
FROM v_ibk as i
INNER JOIN proveedores ON i.proveedor=proveedores.alias
INNER JOIN info ON proveedores.numero_documento=info.ruc
WHERE i.estado='EN PROCESO' ORDER BY i.adquiriente, i.proveedor, i.factura


SELECT
    LPAD(numero_operacion, 7, '0'),
    LPAD(DAY(fecha_operacion), 2, '0') AS dia,
    LPAD(MONTH(fecha_operacion), 2, '0') AS mes,
    YEAR(fecha_operacion) AS ano,
    hora_operacion as hora,
    FORMAT(importe, 2) as importe,
    IF(importe > 9999.99, 'False', 'True') as asterisco,
    IF(proveedor='PALOMINO', 'True', 'False') as palomino,
    IF(proveedor='KENTHIVAS', 'True', 'False') as kenthivas,
    IF(proveedor='CONSULCACH', 'True', 'False') as cach,
    IF(proveedor='CHERRYS', 'True', 'False') as cherrys,
    IF(proveedor='CONSULCELIZ', 'True', 'False') as consulceliz,
    IF(proveedor='NEGORABILLY', 'True', 'False') as negora,
    IF(proveedor='TISOCNI', 'True', 'False') as tisocni,
    IF(proveedor='ESPINO', 'True', 'False') as espinoramos,
    IF(proveedor='ELITE', 'True', 'False') as elitesecurity,
    IF(proveedor='IMPULSAMAS', 'True', 'False') as impulsa,
    IF(proveedor='ENFOCATE', 'True', 'False') as enfocate,
    IF(proveedor='INBOX', 'True', 'False') as inbox,
    IF(proveedor='INVSONIC', 'True', 'False') as mayosonic,
    IF(proveedor='SONICSERV', 'True', 'False') as servsonic,
    IF(proveedor='TOCAM', 'True', 'False') as tocam,
    IF(proveedor='SAYDONE', 'True', 'False') as saydone,
    IF(proveedor='GENZ', 'True', 'False') as genz,
    IF(proveedor='TROYA', 'True', 'False') as troya,
    IF(proveedor='NOVATEX', 'True', 'False') as novax,
    IF(proveedor='VYC', 'True', 'False') as vyc,
    IF(proveedor='CELIZ', 'True', 'False') as carlosceliz,
    IF(proveedor='INGCELIZ', 'True', 'False') as ingceliz,
    IF(proveedor='INGCACH', 'True', 'False') as ingcach,
    IF(proveedor='BLANQUIAZUL', 'True', 'False') as blanquiazul,
    IF(proveedor='MUNRAY', 'True', 'False') as munray,
    IF(proveedor='OLIVEROS', 'True', 'False') as oliveros,
    IF(proveedor='JORA', 'True', 'False') as jora,
    IF(proveedor='SILVER', 'True', 'False') as silver,
    IF(proveedor='JMV', 'True', 'False') as jmv,
    IF(proveedor='IMPULSAOE', 'True', 'False') as impulsaoe,
    IF(proveedor='ELIAPAC', 'True', 'False') as eliapac,
    IF(proveedor='THB', 'True', 'False') as thb,
    IF(proveedor='APOYO', 'True', 'False') as apoyo,
    IF(proveedor='PARJU', 'True', 'False') as parju,
    IF(proveedor='TEXTILES', 'True', 'False') as textiles,
    IF(proveedor='GREJULCA', 'True', 'False') as grejulca
    FROM v_bcp WHERE estado = 'EN PROCESO' ORDER BY adquiriente, proveedor, fecha_operacion, hora_operacion;


**VERIFICAR PEDIDOS BANCARIZADOS**
SELECCIONAR TODOS LOS COMPROBANTES ASOCIADOS DE LAS BANCARIZACIONES Y SUS PROVEEDORES (EMISORES)
BUSCAR RUC DEL PROVEEDOR Y ELABORAR CUI


In [7]:
bancarizado=bcp[['adquiriente', 'proveedor', 'documento_relacionado']]
bancarizado_proveedores=pd.merge(bancarizado, proveedores[['alias', 'numero_documento']],left_on='proveedor', right_on='alias', how='left')
bancarizado_proveedores['numero_documento'] = bancarizado_proveedores['numero_documento'].astype(str)
bancarizado_proveedores['numero_documento'] = bancarizado_proveedores['numero_documento'].apply(lambda x: hex(int(x))[2:])
bancarizado_proveedores['cui'] = bancarizado_proveedores['numero_documento'].astype(str) + '01' + bancarizado_proveedores['documento_relacionado'].astype(str)
bancarizado_proveedores['cui'] = bancarizado_proveedores['cui'].str.replace('-', '')
bancarizado_proveedores=bancarizado_proveedores[['adquiriente', 'proveedor', 'cui']]


In [8]:
bancarizado_proveedores['cui'] = bancarizado_proveedores['cui'].str.lower()
acc_5['cui'] = acc_5['cui'].str.lower()
bancarizado_acc5=pd.merge(bancarizado_proveedores, acc_5[['periodo_tributario', 'fecha_emision', 'numero_documento', 'cui']],on='cui', how='left')


In [9]:
"""
HAY INCONSISTENCIAS Y XMLS NO ANALIZADOS, ELABORAR ANALIZADOR DE XML CON PYTHON Y UNIR A ORGANIZADOR DE ARCHIVOS
"""
bancarizado_filtrado = bancarizado_acc5[~bancarizado_acc5['numero_documento'].isna()]
bancarizado_filtrado['pedido_unido'] = bancarizado_filtrado['periodo_tributario'].astype(int).astype(str) + bancarizado_filtrado['adquiriente'].astype(str)
bancarizado_filtrado = bancarizado_filtrado.drop_duplicates(subset='pedido_unido', keep='first')


C:\Users\Raknaros\AppData\Local\Temp\ipykernel_17980\1141988865.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bancarizado_filtrado['pedido_unido'] = bancarizado_filtrado['periodo_tributario'].astype(int).astype(str) + bancarizado_filtrado['adquiriente'].astype(str)


In [10]:
bancarizado_filtrado.to_excel('bancarizado.xlsx', index=False)

**TRANSFORMAR CUADRO DE VENTAS acc_5 A PEDIDOS**
pedidos_contabilidad=acc_5[acc_5['tipo_documento'] != '0'][['periodo_tributario', 'numero_documento']]
pedidos_contabilidad['pedido_unido'] = pedidos_contabilidad['periodo_tributario'].astype(str) + pedidos_contabilidad['numero_documento'].astype(str)
pedidos_filtrado = pedidos_contabilidad.drop_duplicates(subset='pedido_unido', keep='first')
pedidos_filtrado['numero_documento'] = pedidos_filtrado['numero_documento'].astype(int)
pedidos_inferido=pd.merge(pedidos_filtrado,adquirientes[['related_user', 'alias', 'ruc']],left_on='numero_documento', right_on='ruc', how='left')
